# How to build an AI that can answer questions about your website

https://platform.openai.com/docs/tutorials/web-qa-embeddings

In [1]:
# TODO: https://platform.openai.com/docs/tutorials/web-qa-embeddings
# Maybe https://developer.doordash.com/en-US/docs/welcome
# https://github.com/openai/openai-cookbook/tree/main/apps/web-crawl-q-and-a
# https://github.com/openai/tiktoken
